# Streamflow Data Download and Preprocessing

Metadata for streamflow to be downloaded from: https://wateroffice.ec.gc.ca/station_metadata/station_characteristics_e.html using the specifications `Province = Alberta`, `Parameter Type = Flows`, and `Regulation = Natural` and saved as `station_metadata.csv`.

The stations listed in the metadata file subject to the date specifications are downloaded below from HYDAT and saved to `combined_streamflow.csv`.

In [1]:
import pandas as pd
from src.data_ingestion import fetch_streamflow_batch
from src.config import RAW_DATA_DIR

METADATA_PATH = RAW_DATA_DIR / "station_metadata.csv"
OUTPUT_FILENAME = "combined_streamflow.csv"
START_YEAR = 1980
END_YEAR = 2022

# Filter stations
metadata = pd.read_csv(METADATA_PATH)
filtered_stations = metadata[
    (metadata['Year From'] <= START_YEAR) & 
    (metadata['Year To'] >= END_YEAR)
]["Station Number"].tolist()

# download data
df_streamflow = fetch_streamflow_batch(filtered_stations, START_YEAR, END_YEAR, OUTPUT_FILENAME)

Data downloaded and saved to combined_streamflow.csv
15706 days of data saved for 181 stations


In [2]:
from src.processing import filter_stations_by_annual_completeness
from src.config import RAW_DATA_DIR
import pandas as pd

# 1. Load raw
df_raw = pd.read_csv(RAW_DATA_DIR / "combined_streamflow.csv", index_col="Date", parse_dates=True)

# 2. Filter (Strict: Drop station if any year is >40% missing)
df_filtered = filter_stations_by_annual_completeness(df_raw, max_missing_pct=40.0)
df_filtered.head()

Filtering at 40.0% annual threshold:
 - Keeping 118 stations.
 - Dropping 63 stations due to incomplete years.


,05AA004,05AA008,05AA022,05AA028,05AB005,05AB029,05AD003,05AD035,05AE005,05AH037,...,07JF002,07KE001,07OA001,07OB003,07OB004,07OB006,07OC001,11AA026,11AB117,11AE009
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,NaN,1.01,2.07,NaN,NaN,NaN,2.90,NaN,NaN,NaN,...,NaN,2.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048
1980-01-02,NaN,1.04,2.05,NaN,NaN,NaN,2.85,NaN,NaN,NaN,...,NaN,2.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045
1980-01-03,NaN,1.04,2.04,NaN,NaN,NaN,2.77,NaN,NaN,NaN,...,NaN,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042
1980-01-04,NaN,1.03,2.01,NaN,NaN,NaN,2.69,NaN,NaN,NaN,...,NaN,2.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034
1980-01-05,NaN,1.02,1.98,NaN,NaN,NaN,2.74,NaN,NaN,NaN,...,NaN,2.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023


Water basin polygons can be downloaded from https://collaboration.cmc.ec.gc.ca/cmc/hydrometrics/www/HydrometricNetworkBasinPolygons/gpkg/. The major drainage areas (MDA) selected are:
* (5) Nelson River
* (7) Great Slave Lake

These MDAs were selected due to their proximity to the Eastern Rockies in Alberta.